In [4]:
from input_proc import *
from p1_funs import persist_rets_bin, load_rets_from_bin, floatr_to_uint_minret, uint_ret_offs_tofloatr

import pandas as pd
import numpy as np
from pathlib import Path



In [6]:
# answer to the question: how to store stock returns in a file
# without losing precision best option in terms of size is to use compressed numpy array
# column names and indices can be stored separately, e.g. in another arrays inside compressed container
persist_rets_bin(str(fname) + r'\floatGzipped', xdf)  # 155 MB

In [7]:
# other ways described, some optimal for particular tasks, e.g. columnar storage for PySpark jobs
import pyarrow as pa
import pyarrow.parquet as pq

# Convert pandas DataFrame to PyArrow Table
table = pa.Table.from_pandas(xdf)

# Write PyArrow Table to Parquet file
pq.write_table(table, 'xdf.parquet')  # already less than pkl and only 20% more than np compressed
pq.write_table(table, 'xdfgzipped.parquet', compression='gzip')  # 181MB
# gzip only 3% improvement over uncompressed parquet, zstd same, no improvement with brotli either


# quick search of compressors suggested to use blosc2, but suboptimal as requires C-arrays
import blosc2 as bl2

compressed = bl2.pack_array(np.ascontiguousarray(xarr))  # incl. clvl9 and shuffle

# Save the compressed data to a file, > 200 MB
with open('cr_compressed_blosc.b2frame', 'wb') as f:
    f.write(compressed)

blarr = bl2.compress2(np.ascontiguousarray(xarr))  # fails with Fortran array
# savedsz = bl2.save_array(blarr, str(Path(fname, 'cr_compressed_blosc.bl2')), mode='w')
# this works but floods console, anyway worse than npz 162 MB, so disappointing


In [8]:
# h5py is also a good option, but not as good as compressed numpy array,
# leaving here for reference, listing all tried options
import h5py
with h5py.File(str(Path(fname, 'crDF_compressed9.h5')), "w") as f:
    # same size
    # dset_coefs = f.create_dataset("a", data=xdf, compression="gzip", compression_opts=9)
    dset_coefs = f.create_dataset("a", data=xdf, compression="gzip", compression_opts=9, shuffle=True)
    # dset_coefs = f.create_dataset("a", data=xdf, compression="SZIP", compression_opts=9)
    # dset_coefs = f.create_dataset("a", data=xdf, compression="ZFP", compression_opts=9)
    # dset_coefs = f.create_dataset("a", data=xdf, compression="LZF", compression_opts=9)
    # dset_coefs = f.create_dataset("a", data=xdf, compression="LZMA", compression_opts=9)
# enabling shuffling can potentially improve the compression ratio without modifying the original data
#  after decompression will be the same as the original data. 177 MB, rather slow
#  xdf or xarr (xdf.values) can be passed, no difference in size


In [9]:
# instead of trying more compression tools incl. lossy, we can do our own simplistic one
# reducing precision to float32 would have been too simple:
# but decrease size of nobs * ncomps * 8 bytes for float64 from 232 to 160 MB
persist_rets_bin(Path(fname, 'f32' + 'gzipped'), xdf.astype(np.float32))  # 80MB, half the f64 size as expected


In [12]:
# also, we can use min and max values from data to create custom dtype knowing range of our values:
print(np.nanmax(xarr))
print(np.nanmin(xarr))  # but this is more like data-mining

# more 'theory' solid is to make use of the fact that stock returns are > -1 (price can't go below 0)


# numpy.uint16  # 0 to 65535, whereas uint8 is 0 to 255 (not enough for either company ids or dates)
print(np.iinfo(np.uint16).max)

# 32-bit floating-point values cover range from 1.175494351 * 10^-38 to 3.40282347 * 10^+38,
# which is too much for stock returns, so we may want to limit negative bound to -1 and
# (questionably) sacrifice precision to 4 or 5 digits after the decimal point (in practice,
# it depends on upper bound, theoretically unlimited for stock returns, but with some assumptions
# even np.uint64 with 19 digits will have smaller output size than float64, see at the bottom).

# to sum up, method: use integers to count number of steps (e.g. 0.0001 step size for
#  4 digits after the decimal point) from minimal value, which for stock returns is -1.

# so for above we can use uint16 given our data, but best to use uint32 to be on the safe side:
print(np.nanmin(xarr[xarr>0]))  # > 10**(-10) step size but < even 10**(-9)...
print(np.nanmax(xarr[xarr<0]))  # similarly, > -10**(-9) but < -10**(-10)
print(np.nanmin(np.abs(xarr)))  # equivalent test for both bounds, can be also inferred from f64 type
print(np.nanmax(np.abs(xarr)))


0.06422126285369435
-0.061545515358494424
65535
4.4723612007234463e-10
-1.0699493636284339e-10
1.0699493636284339e-10
0.06422126285369435


In [14]:
# first, use defined function to convert float64 to uint16 per this logic, to show how it works:

print(floatr_to_uint_minret(xarr, prec=5, minval=-0.1)[:10,:10])  # can use for our random data
print('---'*10, ' converted as will be saved with more appropriate defaults for stock returns (more general case)')
print(floatr_to_uint_minret(xarr, prec=4, minval=-1)[:10,:10])


# now, show saving of data:

[[ 9835     0  8630  9695 12496  9223     0 10832     0     0]
 [    0     0     0 10308 10715 10759  9467  9490 10470  7911]
 [    0 10820 10981  9785 10001  8189 10053 10958 10381     0]
 [ 7928     0  9300     0     0 10023 11393 11178     0  9565]
 [ 7575     0  9468     0     0 10476  9989  8515     0  9779]
 [    0     0 10159     0 10390  8105  8592  7607     0 10139]
 [10048 10721  9195 11318 10724  9453  9570 11060 10165  9899]
 [ 8820 12257     0     0     0 11535  7631     0  9576     0]
 [    0  7967 11336  8428  8364 11246 10098     0 10414     0]
 [    0     0 11370 10738 11927     0     0 10207 10651  8474]]
------------------------------  converted as will be saved with more appropriate defaults for stock returns (more general case)
[[ 9984     0  9864  9970 10251  9923     0 10084     0     0]
 [    0     0     0 10032 10072 10077  9948  9950 10048  9792]
 [    0 10083 10099  9979 10001  9820 10006 10097 10039     0]
 [ 9794     0  9931     0     0 10003 10140 10119   

In [15]:
persist_rets_bin(str(fname) + r'\uint16offst', xdf, to_int=np.uint16, prec=4, minval=-1)  # 34 MB is
#  80% reduction in size, but obviously we lose precision, may be outside of the task scope...
persist_rets_bin(Path(fname, 'ui32_p9' + 'gzipped'), xdf, to_int = np.uint32, prec=9)  # 78MB, see below to check precision vs. f32
persist_rets_bin(Path(fname, 'ui64_p19' + 'gzipped'), xdf, to_int = np.uint64, prec=19)  # 146MB

# now showcase restoring values and checking lost precision is as expected (also, covered in unit test):

In [18]:
xdf2 = load_rets_from_bin('uint16offst_npcomprsd.npz', val_dtype=np.uint16, prec=4, minval=-1)  # also converts uint16 to float64

prec = 4
stepsz = 10**(-prec)  # fails with smaller rounding, 3 tests below with increasing "difficulty"

errAssertPrec = "reconstructed values differ from original"
assert np.nanmax(np.abs(xdf.values.round(4) - xdf2.values)) < stepsz, errAssertPrec
assert np.nanmax(np.abs(xdf.values - xdf2.values)) < stepsz, errAssertPrec
assert np.allclose(np.nan_to_num(xdf.values), np.nan_to_num(xdf2.values), atol=1e-4), errAssertPrec

#  may need improvement before production use
print(np.nanmean(np.abs(xdf.values - xdf2.values)),  'average error')
print(np.nanmax( np.abs(xdf.values - xdf2.values)), 'worst error')

# but both within tolerance level:
print((np.nanmax( np.abs(xdf.values - xdf2.values)) > np.nanmean( np.abs(xdf.values - xdf2.values)))
        and (np.nanmax( np.abs(xdf.values - xdf2.values)) < stepsz))

2.5002338192945866e-05 average error
4.9999995246557266e-05 worst error
True


In [19]:
# ui32 9-digit precision vs. f32 (only 2MB larger size for f32, but 4x worse precision):
xarr2_f32 = np.load(file=Path(fname, 'f32' + 'gzipped' + '_npcomprsd.npz'))['arr']
xdf2_ui32 = load_rets_from_bin(Path(fname, 'ui32_p9' + 'gzipped' + '_npcomprsd.npz'),
                               val_dtype=np.uint32, prec=9, minval=-1)

# beats on max abs error, but not on mean abs error; depends on user or use case which is more important
print(np.nanmax(np.abs(xdf.values - xarr2_f32)), 'Max Abs Err for f32')
print(np.nanmax(np.abs(xdf.values - xdf2_ui32.values)), 'Max Abs Err for ui32')


1.8626446982028533e-09 Max Abs Err for f32
5.000001385147002e-10 Max Abs Err for ui32
2.0565834790162216e-10 MAE for f32
2.500288090765228e-10 MAE for ui32


average error better for f32, but not by much (20%) vs 4x for max abs error (where ui32 is better)

In [20]:
print(np.nanmean(np.abs(xdf.values - xarr2_f32)), 'MAE for f32')
print(np.nanmean(np.abs(xdf.values - xdf2_ui32.values)), 'MAE for ui32')

2.0565834790162216e-10 MAE for f32
2.500288090765228e-10 MAE for ui32


function now also works with not just min value and precision specified, but also a range of values min and max, (auto-inferring precision), which may be useful in some cases


also, test range functionality, mapping e.g. floats between values [-1, 1] to ints [0, 65535] assumption of fixed step

In [21]:
xret3rng = floatr_to_uint_minret(xarr.copy(), minval=-0.07, maxval=0.07)
xret3_orng = uint_ret_offs_tofloatr(xret3rng, minval=-0.07, maxval=0.07)
print(np.nanmean(np.abs(xarr.round(4) - xret3_orng)))  # strangely on range precision is worse than on minret only, same with nanmax
print(np.nanmax(np.abs(xarr.round(4) - xret3_orng)))  # 0.0001


xret3rngu = floatr_to_uint_minret(xarr.copy(), prec=4, minval=-1)
xret3_orngu = uint_ret_offs_tofloatr(xret3rngu, prec=4, minval=-1)

print(np.nanmean(np.abs(xarr.round(4) - xret3_orngu)))  # indeed, yields smallest error, same with nanmax
print(np.nanmax(np.abs(xarr.round(4) - xret3_orngu)))  # 0.0001, TBD, debugging/fix outside of task scope

2.500605056077757e-05
5.1057466353355424e-05
4.1627594848535255e-17
1.1102230246251565e-16
